In [1]:
# Import libraries
import pandas as pd
import networkx as nx

# Load the data
data = pd.read_excel("C:/Users/timot/Documents/Master data science for business and entrepreneurship/Social network analysis/BACRIM2020-DB.xlsx")
print(data.columns)
# Combine rows with the same 'grupo' by concatenating alliances and setting binary columns
data_combined = (
    data.groupby('grupo').agg({
        'estado': lambda x: ";".join(pd.Series(";".join(x.dropna()).split(";")).unique()), # Concatenate unique estados
        'aliados': lambda x: ";".join(pd.Series(";".join(x.dropna()).split(";")).unique()),  # Concatenate unique allies
        'rivales': lambda x: ";".join(pd.Series(";".join(x.dropna()).split(";")).unique()),  # Concatenate unique rivals
        'persona': lambda x: ";".join(x.dropna()),  # Concatenate persona information if it exists
        'número de rivales': lambda x: int(any(x == 1)),  # Binary column
        'actividades_delictivas': lambda x: int(any(x == 1)),
        'narcotrafico': lambda x: int(any(x == 1)),
        'conflictos_armados': lambda x: int(any(x == 1)),
        'presencia_noviolenta': lambda x: int(any(x == 1)),
        'accion_guber': lambda x: int(any(x == 1)),
        'otros': lambda x: int(any(x == 1))
    }).reset_index()
)


# Prepare edges (alliances and rivalries) with unique 'grupo' identifiers
edges = []
for i, row in data_combined.iterrows():
    cartel = row['grupo']
    
    # Process allies
    if pd.notna(row['aliados']):
        allies = row['aliados'].split(";")
        for ally in allies:
            edges.append({'from': cartel, 'to': ally, 'type': 'alliance'})
    
    # Process rivals
    if pd.notna(row['rivales']):
        rivals = row['rivales'].split(";")
        for rival in rivals:
            edges.append({'from': cartel, 'to': rival, 'type': 'rivalry'})

# Convert edges list to a DataFrame and remove duplicates
print(edges)
edges_df = pd.DataFrame(edges).drop_duplicates()
print(edges_df)

# Sort 'from' and 'to' columns lexicographically for each row so (A, B) and (B, A) are treated as the same
edges_df[['from_sorted', 'to_sorted']] = edges_df.apply(lambda x: sorted([x['from'], x['to']]), axis=1, result_type='expand')

# Drop duplicates based on the sorted columns ('from_sorted' and 'to_sorted') to ensure undirected edge uniqueness
valid_edges = edges_df.drop_duplicates(subset=['from_sorted', 'to_sorted'])

# Drop the sorting columns after deduplication to get back the original 'from' and 'to' structure
valid_edges = valid_edges[['from', 'to', 'type']]

# Filter edges to ensure 'from' and 'to' vertices exist in `data_combined`
valid_edges = valid_edges[
    valid_edges['from'].isin(data_combined['grupo']) &
    valid_edges['to'].isin(data_combined['grupo'])
]

# Export the valid_edges DataFrame to a CSV file
valid_edges.to_csv("valid_edges.csv", index=False)
data_combined.drop(columns = ["aliados", "rivales"]).to_csv("vertice_data.csv", index=False)



Index(['id', 'grupo', 'persona', 'estado', 'aliados', 'número de aliados',
       'rivales', 'número de rivales', 'actividades_delictivas',
       'narcotrafico', 'conflictos_armados', 'presencia_noviolenta',
       'accion_guber', 'otros', 'fecha', 'link1', 'link2', 'link3', 'link4',
       'link5', 'link6', 'link7', 'link8', 'link9', '...25', '...26'],
      dtype='object')
[{'from': 'ACME', 'to': 'Cártel Jalisco Nueva Generación (CJNG)', 'type': 'alliance'}, {'from': 'ACME', 'to': 'La Unión Tepito', 'type': 'alliance'}, {'from': 'ACME', 'to': 'Cártel de Tláhuac', 'type': 'alliance'}, {'from': 'ACME', 'to': '', 'type': 'rivalry'}, {'from': 'Banda de El Perro', 'to': '', 'type': 'alliance'}, {'from': 'Banda de El Perro', 'to': '', 'type': 'rivalry'}, {'from': 'CJNG Comando la Mancha', 'to': 'Cártel Jalisco Nueva Generación (CJNG)', 'type': 'alliance'}, {'from': 'CJNG Comando la Mancha', 'to': 'Cártel de Sinaloa', 'type': 'rivalry'}, {'from': 'Caballeros Templarios', 'to': 'La Nueva Fa

In [8]:
import pandas as pd
import numpy as np

# Load the data
data = pd.read_excel("C:/Users/timot/Documents/Master data science for business and entrepreneurship/Social network analysis/BACRIM2020-DB.xlsx")

# Ensure the 'estado' column contains the states in a format that can be split
data['estado'] = data['estado'].fillna('')  # Fill any NaN with an empty string

# One-hot encode the 'estado' column for each unique group
# Assuming the delimiter is ';' - adjust if needed
states_expanded = data[['grupo', 'estado']].drop_duplicates()  # Keep only unique combinations
states_expanded = states_expanded.set_index('grupo')['estado'].str.get_dummies(sep=';')

# Combine one-hot encoded states with unique groups
data = states_expanded.groupby('grupo').max().reset_index()

# Isolate the state presence columns and cartel names
cartel_names = data['grupo']
state_data = data.drop(columns=['grupo'])

# Create an empty dependency matrix
dependency_matrix = np.zeros((len(cartel_names), len(cartel_names)), dtype=int)

# Compute the dependency matrix
for i in range(len(cartel_names)):
    for j in range(len(cartel_names)):
        # Check if there is any common state operation
        if (state_data.iloc[i] & state_data.iloc[j]).any():
            dependency_matrix[i, j] = 1

np.fill_diagonal(dependency_matrix, 0)

# Convert to a DataFrame for readability, with cartels as row and column labels
dependency_df = pd.DataFrame(dependency_matrix, index=cartel_names, columns=cartel_names)

# Save the result to an Excel file
output_path = 'state_dependency_matrix.csv'
dependency_df.to_csv(output_path, index=False)

print(f"File saved as '{output_path}'")
dependency_df

File saved as 'state_dependency_matrix.csv'


grupo,ACME,Banda de El Perro,CJNG Comando la Mancha,Caballeros Templarios,Comandante Coronavirus,Cártel Independiente de Acapulco,Cártel Jalisco Nueva Generación (CJNG),Cártel Los Epitacio,Cártel Nueva Plaza,Cártel Pura Gente Nueva,...,Pedro Aviles Salazar,Robles,Sangre Nueva Zeta,Sindicato Libertad,Tercera Acción Destructiva (3AD),Tláhuac-Barbas,Tropa del Infierno,U o la UVA,Unión de Pueblos y Organizaciones del Estado de Guerrero (UPOEG),Zetas Vieja Escuela
grupo,,,,,,,,,,,,,,,,,,,,,
ACME,0,1,0,1,0,0,1,0,0,0,...,0,1,0,1,1,1,0,1,0,0
Banda de El Perro,1,0,0,1,0,0,1,0,0,0,...,0,1,0,1,1,1,0,1,0,0
CJNG Comando la Mancha,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Caballeros Templarios,1,1,0,0,0,0,1,0,0,0,...,0,1,0,1,1,1,0,1,0,0
Comandante Coronavirus,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tláhuac-Barbas,1,1,0,1,0,0,1,0,0,0,...,0,1,0,1,1,0,0,1,0,0
Tropa del Infierno,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
U o la UVA,1,1,0,1,0,0,1,0,0,0,...,0,1,0,1,1,1,0,0,0,0
